In [ ]:
import time
from pyspark.sql.connect.session import SparkSession
%run ./setup.ipynb
%run ./history.ipynb
%run ./bronze.ipynb
%run ./silver.ipynb
%run ./gold.ipynb

In [ ]:
if __name__ == "__main__":
    spark = (
        SparkSession.builder
        .appName("GymWorkoutRun")
        .enableHiveSupport()
        .getOrCreate()
    )
    sc = spark.sparkContext
    spark.conf.set("spark.sql.shuffle.partitions", sc.defaultParallelism)
    spark.conf.set("spark.databricks.delta.optimizeWrite.enabled", True)
    spark.conf.set("spark.databricks.delta.autoCompact.enabled", True)
    spark.conf.set("spark.sql.streaming.stateStore.providerClass", "com.databricks.sql.streaming.state.RocksDBStateStoreProvider")
    env = "dev"
    once = True
    processing_time = "5 seconds"

    sh = SetupHelper(spark, env)
    hl = HistoryLoader(spark, env)

    setup_required = spark.sql(f"SHOW DATABASES IN {sh.catalog}").filter(f"databaseName == '{sh.db_name}'").count() != 1
    if setup_required:
        sh.setup()
        hl.load_history()
    else:
        spark.sql(f"USE {sh.catalog}.{sh.db_name}")

    bz = Bronze(spark, env)
    sl = Silver(spark, env)
    gl = Gold(spark, env)

    bz.consume(once, processing_time)
    sl.upsert(once, processing_time)
    gl.upsert(once, processing_time)